In [1]:
# ==================================================================================================== Step 1: Import Libraries and Load Images
import cv2
import numpy as np

# Set the path to the images
img = [
    "img/no (1).jpg",
    "img/no (2).jpg",
    "img/no (3).jpg",
    "img/no (4).jpg",
    "img/no (5).jpg",
    "img/no (6).jpg",
    "img/no (7).jpg",
    "img/no (8).jpg",
    "img/no (9).jpg",
    "img/no (10).jpg",
    "img/gg (1).jpg",
    "img/gg (2).jpg",
    "img/gg (3).jpg",
    "img/gg (4).jpg",
    "img/m (1).jpg",
    "img/m (2).jpg",
    "img/m (3).jpg",
    "img/m (4).jpg",
    "img/m (5).jpg",
    "img/m (6).jpg",
]

# Shuffle the images
np.random.shuffle(img)

# Prepare the correct answers
correct_ans = []
for i in img:
    correct_ans.append("No" if "no" in i else "Yes")

# Load the images in grayscale
df_img = []
for i in img:
    df_img.append(cv2.imread(i, cv2.IMREAD_GRAYSCALE))

In [2]:
# ==================================================================================================== Step 2: Preprocess the Images
# Crop image of unwanted padding
cropped_img = []
for img in df_img:
    # Threshold the image
    threshold = cv2.threshold(img.copy(), 40, 255, cv2.THRESH_OTSU)[1]
    # Find the contours
    contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Get largest contour
    c = max(contours, key=cv2.contourArea)
    # Get extreme points of the contour
    extleft = tuple(c[c[:, :, 0].argmin()][0])
    extright = tuple(c[c[:, :, 0].argmax()][0])
    exttop = tuple(c[c[:, :, 1].argmin()][0])
    extbot = tuple(c[c[:, :, 1].argmax()][0])
    # Crop the image
    cropped_img.append(img[exttop[1] : extbot[1], extleft[0] : extright[0]])

# Show the cropped images
index = 0
for img in cropped_img:
    cv2.imwrite(f"output/1 Cropped Image {index+1}.jpg", img)
    #     cv2.imshow("Cropped Image", img)
    #     cv2.waitKey(0)
    index += 1
# cv2.destroyAllWindows()

In [3]:
# ==================================================================================================== Step 3: Extract Features and Classify
# Switch to save the result
save_result = True

# 1. Threshold
threshold_img = []
# Parameters
t_val = 100
index = 0
for img in cropped_img:
    _, tres = cv2.threshold(img.copy(), t_val, 255, cv2.THRESH_BINARY)
    cv2.imwrite(f"output/2 Threshold Image {index+1}.jpg", tres)
    threshold_img.append(tres)
    index += 1


# 2. Erode and Dilate
ed_img = []
# Parameters
k_size = 3
kernel = np.ones((k_size, k_size), np.uint8)
total_iterations = 2
e_iterations = 2
d_iterations = 3

index = 0
for img in threshold_img:
    for i in range(total_iterations):
        erode = cv2.erode(img.copy(), kernel, iterations=e_iterations)
        dilate = cv2.dilate(erode, kernel, iterations=d_iterations)
    cv2.imwrite(f"output/3 Eroded and Dilated Image {index+1}.jpg", dilate)
    ed_img.append(dilate)
    index += 1

# 3. Finding Contours, Draw Contours, Extract features
output_img = []
output_ans = []

# Font
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.5
font_color = (255, 255, 255)
font_thickness = 1
text_origin_pred = (10, 20)
text_origin_ans = (10, 50)
text_origin_accurate = (100, 20)

# Output Bar Size
bar_height = 60
bar_width = 150

# Save result img
save_result = True

# Parameters
t_min_area = 0.005
t_max_area = 0.20
t_rect_ratio = 2  # Good Value 2
t_box_pos = 0.15
t_white_1 = 0.25  # 0.25, 0.3 Interesting Result
t_white_2 = 0.35  # 0.35, 0.4 Interesting Result


# Function to add a top bar to the image
def img_add_top_bar(img, height: int, width: int):
    orig_height, orig_width = img.shape
    # Calculate the new image dimensions, ensuring the top bar fits
    new_height = height + orig_height
    new_width = max(orig_width, width)

    # Create a new image with the expanded height and width
    new_img = np.zeros((new_height, new_width), dtype=np.uint8)
    # Place the original image below the top bar
    new_img[height:, :orig_width] = img
    return new_img


index = 0
for img in ed_img:
    # Get the index of the image
    original_img = cropped_img[index]
    correct_answer = correct_ans[index]

    # Initialize the variables
    predStr = ""
    outputAns = ""
    correct_or_wrong_str = ""
    new_canvas = original_img.copy()
    box_count = 0

    # Filter 1: Overall White Pixel Percentage too High
    if np.sum(img == 255) / (img.shape[0] * img.shape[1]) > t_white_1:
        box_count = 0
    else:
        # Find the contours
        contours, _ = cv2.findContours(img, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

        # Filter 2: Base on contour properties
        for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)
            min_area = t_min_area * img.shape[0] * img.shape[1]
            max_area = t_max_area * img.shape[0] * img.shape[1]
            area = w * h
            bound_center = (x + w / 2, y + h / 2)
            low_x = t_box_pos * img.shape[1]
            low_y = t_box_pos * img.shape[0]
            up_x = img.shape[1] - low_x
            up_y = img.shape[0] - low_y
            if min_area < area < max_area and w / h < t_rect_ratio and h / w < t_rect_ratio and (up_x > bound_center[0] > low_x) and (up_y > bound_center[1] > low_y) and (np.sum(img[y : y + h, x : x + w] == 255) / area) > t_white_2:
                cv2.rectangle(new_canvas, (x, y), (x + w, y + h), (255, 255, 255), 1)
                box_count += 1

    # Tally the results
    new_canvas = img_add_top_bar(new_canvas, bar_height, bar_width)
    if box_count > 0:
        predStr = "Pred:Yes"
        outputAns = "Yes"
        correct_or_wrong_str = "Correct" if correct_answer == "Yes" else "Wrong"
    else:
        predStr = "Pred:No"
        outputAns = "No"
        correct_or_wrong_str = "Correct" if correct_answer == "No" else "Wrong"

    # Add labels
    cv2.putText(new_canvas, predStr, text_origin_pred, font, font_scale, font_color, font_thickness, cv2.LINE_AA)
    cv2.putText(new_canvas, "Ans:" + correct_answer, text_origin_ans, font, font_scale, font_color, font_thickness, cv2.LINE_AA)
    cv2.putText(new_canvas, correct_or_wrong_str, text_origin_accurate, font, font_scale, font_color, font_thickness, cv2.LINE_AA)

    # Save the output
    output_img.append(new_canvas)
    output_ans.append(outputAns)
    print("Image ", index + 1, ": Prediction: ", outputAns, " | Answer: ", correct_answer)

    # Save the result
    if save_result:
        cv2.imwrite("output/4 Result " + str(index + 1) + " - " + str(correct_or_wrong_str) + ".jpg", new_canvas)

    index += 1


# ==================================================================================================== Step 4: Display the Results
# Get the accuracy
correct = 0
for i in range(len(output_ans)):
    if output_ans[i] == correct_ans[i]:
        correct += 1
accuracy = correct / len(output_ans) * 100
print("Total Correct: ", correct, " out of ", len(output_ans))
print("Accuracy: ", accuracy, "%")
# Display the output
for i in range(len(output_img)):
    cv2.imshow("Output " + str(i + 1), output_img[i])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Image  1 : Prediction:  No  | Answer:  No
Image  2 : Prediction:  No  | Answer:  No
Image  3 : Prediction:  No  | Answer:  No
Image  4 : Prediction:  No  | Answer:  No
Image  5 : Prediction:  No  | Answer:  No
Image  6 : Prediction:  Yes  | Answer:  Yes
Image  7 : Prediction:  Yes  | Answer:  Yes
Image  8 : Prediction:  Yes  | Answer:  Yes
Image  9 : Prediction:  No  | Answer:  Yes
Image  10 : Prediction:  Yes  | Answer:  No
Image  11 : Prediction:  Yes  | Answer:  Yes
Image  12 : Prediction:  No  | Answer:  No
Image  13 : Prediction:  Yes  | Answer:  Yes
Image  14 : Prediction:  Yes  | Answer:  Yes
Image  15 : Prediction:  No  | Answer:  No
Image  16 : Prediction:  Yes  | Answer:  Yes
Image  17 : Prediction:  No  | Answer:  Yes
Image  18 : Prediction:  No  | Answer:  No
Image  19 : Prediction:  Yes  | Answer:  Yes
Image  20 : Prediction:  No  | Answer:  No
Total Correct:  17  out of  20
Accuracy:  85.0 %
